In [8]:
# Check python version, and path of python executable
import sys
print(sys.version)
print(sys.executable)

# Import and load necessary libraries
import ee as ee
ee.Initialize()
#import ee.mapclient
import pandas as pd

# Print earth engine version (this code works with ee version 0.1.80)
print ee.__version__

#Load datasets
minnesota = ee.FeatureCollection('ft:1fRY18cjsHzDgGiJiS2nnpUU3v9JPDc2HNaR7Xk8').filterMetadata('Name', 'equals', 'Minnesota')
modis = ee.ImageCollection('MODIS/055/MOD17A3')
units = ee.FeatureCollection('ft:1gdhQYMZe31QgEbrmHArQUECKdVssmvR2tqhM03_i')

# Print band names in Modis data
print('Band names in Modis data:', ee.Image(modis.first()).bandNames().getInfo())

# Select NPP band from Modis data, and clip with Minnesota shape
npp = modis.select('Npp')
def clipped(img):
    return img.clip(minnesota)
npp_minnesota = npp.map(clipped)

# Select only a certain year from the data
#year_map = npp_minnesota.filter(ee.Filter.calendarRange(2012, 2012, 'year'));

# Calculate mean npp across all years in the collection (2000-2014)
mean_npp_minnesota = npp_minnesota.reduce(ee.Reducer.mean())

# Reproject mean npp. This is not necessary, but is here for reference
#mean_npp_minnesota_utm15 = ee.Image(mean_npp_minnesota).reproject('EPSG:26915', None, 1000)
#print(mean_npp_minnesota_utm15.projection().nominalScale().getInfo());
#print(mean_npp_minnesota_utm15.projection().crs().getInfo());

# Calculate bounds of the state of Minnesota, used for saving the image
#minnesota_bounds = minnesota.geometry().bounds().getInfo()['coordinates'][0][0:4]
#
# We have two options for downloading mean NPP in Minnesota. We can either 
# save it to the Google drive, or create a download link. The Google 
# drive option seems to work more consistently, and includes nodata 
# pixels (as opposed to setting them to zero).
# 1. Save to Google drive
#task_config = {'description':'mean_npp_minnesota', 
#               'scale':1000, 
#               'crs':'EPSG:26915', 
#               'region': minnesota_bounds}
#task = ee.batch.Export.image(ee.Image(mean_npp_minnesota), 'mean_npp_minnesota', task_config)
#task.start()
#
# 2. Create download link
#path_config = {'description':'mean_npp_minnesota', 
#    'scale': 100,
#    'crs': 'EPSG:26915', 
#    'region': minnesota_bounds}
#path = ee.Image(mean_npp_minnesota).getDownloadUrl(path_config)
#print path

# Calculate mean npp per unit
mean_npp_unit = mean_npp_minnesota.reduceRegions(units, ee.Reducer.mean(), 1000)

# Once again we have two options for downloading the ready data. 
# There is not so much difference between the two options.
# 1. Save to Google drive
#task_config = {'description': 'Mean_npp_per_unit', 
#                'fileFormat': 'csv'}
#task = ee.batch.Export.table(mean_npp_unit, 'mean_npp_unit', task_config)
#task.start()
#
# 2. Create download link
path = mean_npp_unit.getDownloadURL('csv')

# Read data 
df = pd.read_csv(path)
df.rename(columns={'system:index': 'unit', 'mean': 'npp'}, inplace=True)
df[[0, 2]].to_csv('../data/npp/mean_npp_unit.csv')

2.7.11 |Anaconda custom (x86_64)| (default, Dec  6 2015, 18:57:58) 
[GCC 4.2.1 (Apple Inc. build 5577)]
/Users/Eric/anaconda/envs/python27/bin/python
0.1.80
('Band names in Modis data:', [u'Gpp', u'Npp', u'Gpp_Npp_QC'])
111319.490793
EPSG:4326
